# 2 - Fluid particles

Topics covered in this tutorial:

- basic functionalities of [ParticlesSPH](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#base-modules) class
- initializing velocities as $\mathbf v(0) = \mathbf u(\mathbf x(0))$ via a [GenericFluidEquilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#generic-fluid-equilibria)
- velocity push with [PushVinEfield](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVinEfield)
- velocity push with [PushVinSPHpressure](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVinSPHpressure)

## Fluid flow in external force field

Let $\Omega \subset \mathbb R^3$ be a box (cuboid). We search for trajectories $(\mathbf x_p, \mathbf v_p): [0,T] \to \Omega \times \mathbb R^3$, $p = 0, \ldots, N-1$ that satisfy

$$
\begin{align}
 \dot{\mathbf x}_p &= \mathbf v_p\,,\qquad && \mathbf x_p(0) = \mathbf x_{p0}\,,
 \\[2mm]
 \dot{\mathbf v}_p &= -\nabla p(\mathbf x_p) \qquad && \mathbf v_p(0) = \mathbf u(\mathbf x_p(0))\,,
 \end{align}
$$

where $p \in H^1(\Omega)$ is some given function.
In Struphy, the position coordinates are updated in logical space $[0, 1]^3 = F^{-1}(\Omega)$, for instance with the Propagator [PushEta](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushEta) which we shall use in what follows.

In [ ]:
from struphy.geometry.domains import Cuboid

l1 = -.5
r1 = .5
l2 = -.5
r2 = .5
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
from struphy.fields_background.generic import GenericCartesianFluidEquilibrium
import numpy as np

def u_fun(x, y, z):
    ux = -np.cos(np.pi*x)*np.sin(np.pi*y)
    uy = np.sin(np.pi*x)*np.cos(np.pi*y)
    uz = 0 * x 
    return ux, uy, uz

p_fun = lambda x, y, z: 0.5*(np.sin(np.pi*x)**2 + np.sin(np.pi*y)**2)
n_fun = lambda x, y, z: 1. + 0*x

bel_flow = GenericCartesianFluidEquilibrium(u_xyz=u_fun, p_xyz=p_fun, n_xyz=n_fun)
bel_flow.domain = domain
p_xyz = bel_flow.p_xyz
p0 = bel_flow.p0

In [ ]:
from struphy.pic.particles import ParticlesSPH

Np = 1000
bc = ['reflect', 'reflect', 'periodic']

# instantiate Particle object
particles = ParticlesSPH(
        Np=Np,
        bc=bc,
        domain=domain,
        bckgr_params=bel_flow,
    )

In [ ]:
particles.draw_markers(sort=False)
particles.apply_kinetic_bc()
particles.initialize_weights()

In [ ]:
particles.positions
# positions on the physical domain Omega
pushed_pos = domain(particles.positions).T
pushed_pos

In [ ]:
from struphy.propagators.propagators_markers import PushEta

# default parameters of Propagator
opts_eta = PushEta.options(default=False)
print(opts_eta)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [ ]:
# instantiate Propagator object
prop_eta = PushEta(particles, algo = "forward_euler")

In [ ]:
from struphy.feec.psydac_derham import Derham

Nel = [64, 64, 1]  # Number of grid cells
p = [3, 3, 1]  # spline degrees
spl_kind = [False, False, True]   # spline types (clamped vs. periodic)

derham = Derham(Nel, p, spl_kind)

In [ ]:
p_coeffs = derham.P["0"](p0)
p_coeffs

In [ ]:
from struphy.propagators.propagators_markers import PushVinEfield

# instantiate Propagator object
PushVinEfield.domain = domain
PushVinEfield.derham = derham

In [ ]:
p_h = derham.create_field('pressure', 'H1')
p_h.vector = p_coeffs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 12))
x = np.linspace(-.5, .5, 100)
y = np.linspace(-.5, .5, 90)
xx, yy = np.meshgrid(x, y)
eta1 = np.linspace(0, 1, 100)
eta2 = np.linspace(0, 1, 90)

plt.subplot(2, 2, 1)
plt.pcolor(xx, yy, p_xyz(xx, yy, 0))
plt.axis('square')
plt.title('p_xyz')
plt.colorbar()

plt.subplot(2, 2, 2)
p_vals = p0(eta1, eta2, 0, squeeze_out=True).T
plt.pcolor(eta1, eta2, p_vals)
plt.axis('square')
plt.title('p logical')
plt.colorbar()

plt.subplot(2, 2, 3)
p_h_vals = p_h(eta1, eta2, 0, squeeze_out=True).T
plt.pcolor(eta1, eta2, p_h_vals)
plt.axis('square')
plt.title('p_h (logical)')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.pcolor(eta1, eta2, np.abs(p_vals - p_h_vals))
plt.axis('square')
plt.title('difference')
plt.colorbar()

In [ ]:
grad_p = derham.grad.dot(p_coeffs)
grad_p.update_ghost_regions() # very important, we will move it inside grad
grad_p *= -1.
prop_v = PushVinEfield(particles, e_field=grad_p)

In [ ]:
import numpy as np
import math
import tqdm

# time stepping
dt = 0.02
Nt = 200

pos = np.zeros((Nt + 1, Np, 3), dtype=float)
velo = np.zeros((Nt + 1, Np, 3), dtype=float)
energy = np.zeros((Nt + 1, Np), dtype=float)

particles.draw_markers(sort=False)
particles.apply_kinetic_bc()
particles.initialize_weights()

pos[0] = domain(particles.positions).T
velo[0] = particles.velocities
energy[0] = .5*(velo[0, : , 0]**2 + velo[0, : , 1]**2) + p_h(particles.positions)

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0
while n < Nt:
    time += dt
    n += 1
    time_vec[n] = time
    
    # advance in time
    prop_eta(dt/2)
    prop_v(dt)
    prop_eta(dt/2)
    
    # positions on the physical domain Omega
    pos[n] = domain(particles.positions).T
    velo[n] = particles.velocities
    
    energy[n] = .5*(velo[n, : , 0]**2 + velo[n, : , 1]**2) + p_h(particles.positions)

In [ ]:
# energy plots
fig = plt.figure(figsize = (13, 6))

plt.subplot(2, 2, 1)
plt.plot(time_vec, energy[:, 0])
plt.title('particle 1')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 2)
plt.plot(time_vec, energy[:, 1])
plt.title('particle 2')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 3)
plt.plot(time_vec, energy[:, 2])
plt.title('particle 3')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 4)
plt.plot(time_vec, energy[:, 3])
plt.title('particle 4')
plt.xlabel('time')
plt.ylabel('energy')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter(pos[-1,:,0],pos[-1,:,1],pos[-1,:,2])

In [ ]:
plt.figure(figsize=(12, 28))

coloring = np.select([pos[0,:,0]<=-0.2, np.abs(pos[0,:,0]) < +0.2, pos[0,:,0] >= 0.2],
                        [-1.0, 0.0, +1.0])

interval = Nt/20
plot_ct = 0
for i in range(Nt):
    if i % interval == 0:
        print(f'{i = }')
        plot_ct += 1
        plt.subplot(5, 2, plot_ct)
        ax = plt.gca() 
        plt.scatter(pos[i, :, 0], pos[i, :, 1], c=coloring)
        plt.axis('square')
        plt.title('n0_scatter')
        plt.xlim(l1, r1)
        plt.ylim(l2, r2)
        plt.colorbar()
        plt.title(f'Gas at t={i*dt}')
    if plot_ct == 10:
        break

In [ ]:
make_movie = False
if make_movie:
    import matplotlib.animation as animation
    n_frame = Nt
    fig, ax = plt.subplots()

    coloring = np.select([pos[0,:,0]<=-0.2, np.abs(pos[0,:,0]) < +0.2, pos[0,:,0] >= 0.2],
                        [-1.0, 0.0, +1.0])
    scat = ax.scatter(pos[0,:,0], pos[0,:,1], c=coloring)
    ax.set_xlim([-0.5,0.5])
    ax.set_ylim([-0.5,0.5])
    ax.set_aspect('equal')

    f = lambda x, y: np.cos(np.pi*x)*np.cos(np.pi*y)
    ax.contour(xx, yy, f(xx, yy))
    ax.set_title(f'time = {time_vec[0]:4.2f}')

    def update_frame(frame):
        scat.set_offsets(pos[frame,:,:2])
        ax.set_title(f'time = {time_vec[frame]:4.2f}')
        return scat

    ani = animation.FuncAnimation(fig=fig, func=update_frame, frames = n_frame)
    ani.save("tutorial_02_movie.gif")

## Gas expansion

We use SPH to solve Euler's equations

$$
\begin{align}
 \partial_t \rho + \nabla \cdot (\rho \mathbf u) &= 0\,,
 \\[2mm]
 \rho(\partial_t \mathbf u + \mathbf u \cdot \nabla \mathbf u) &= - \nabla \left(\rho^2 \frac{\partial \mathcal U(\rho, S)}{\partial \rho} \right)\,,
 \\[2mm]
 \partial_t S + \mathbf u \cdot \nabla S &= 0\,,
 \end{align}
$$

where $S$ denotes the entropy per unit mass and the internal energy per unit mass is 

$$
\mathcal U(\rho, S) = \kappa(S) \log \rho\,.
$$

The SPH discretization leads to ODEs for $N$ particles indexed by $p$,

$$
\begin{align}
 \dot{\mathbf x}_p &= \mathbf v_p\,,\qquad && \mathbf x_p(0) = \mathbf x_{p0}\,,
 \\[2mm]
 \dot{\mathbf v}_p &= -\kappa_{p}(0) \sum_{q=1}^N w_p w_q \left(\frac{1}{\rho^{N,h}(\mathbf x_p)} + \frac{1}{\rho^{N,h}(\mathbf x_q)} \right) \nabla W_h(\mathbf x_p - \mathbf x_q) \qquad && \mathbf v_p(0) = \mathbf u(\mathbf x_p(0))\,,
 \end{align}
$$

where the smoothed density reads

$$
 \rho^{N,h}(\mathbf x) = \sum_{p=1}^N w_p W_h(\mathbf x - \mathbf x_p)\,,
$$

with weights $w_p = const.$ and where $W_h(\mathbf x)$ is a suitable smoothing kernel.
The velocity update is performed with the Propagator [PushVinSPHpressure](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVinSPHpressure).

In [ ]:
from struphy.geometry.domains import Cuboid

l1 = -3
r1 = 3
l2 = -3
r2 = 3
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
from struphy.fields_background.generic import GenericCartesianFluidEquilibrium
import numpy as np
T_h = 0.2
gamma = 5/3
n_fun = lambda x, y, z: np.exp(-(x**2 + y**2)/T_h)

bckgr = GenericCartesianFluidEquilibrium(n_xyz=n_fun)
bckgr.domain = domain

In [ ]:
#particle initialization 
from struphy.pic.particles import ParticlesSPH

# marker parameters
ppb = 400
nx = 16
ny = 16
nz = 1
boxes_per_dim = (nx, ny, nz)
bc = ['periodic']*3

# instantiate Particle object
particles = ParticlesSPH(
        #Np=10000,
        ppb=ppb,
        boxes_per_dim=boxes_per_dim,
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        verbose_boxes=False,
    )

In [ ]:
threshold = 1e-1
particles.draw_markers(sort=False)
particles.apply_kinetic_bc()
particles.initialize_weights(reject_weights=True, threshold=threshold)

In [ ]:
particles.markers.shape

In [ ]:
particles.sorting_boxes.boxes.shape

In [ ]:
components = [True, True, False, False, False, False]
nx_b = 64
ny_b = 64
be_x = np.linspace(0, 1, nx_b + 1)
be_y = np.linspace(0, 1, ny_b + 1)
bin_edges = [be_x, be_y]
f_bin, df_bin = particles.binning(components, bin_edges, divide_by_jac=False)
f_bin.shape

In [ ]:
x = np.linspace(l1, r1, 100)
y = np.linspace(l2, r2, 90)
xx, yy = np.meshgrid(x, y)
eta1 = np.linspace(0, 1, 100)
eta2 = np.linspace(0, 1, 90)
eta3 = np.linspace(0,1,1)
ee1, ee2, ee3 = np.meshgrid(eta1, eta2, eta3)

In [ ]:
kernel_type = "gaussian_2d" 
h1 = 1/nx
h2 = 1/ny
h3 = 1/nz

n_sph = particles.eval_density(ee1, ee2, ee3, h1=h1, h2=h2, h3=h3, kernel_type=kernel_type, fast=True,)
n_sph.shape

In [ ]:
logpos = particles.positions
weights = particles.weights
print(f'{logpos.shape = }')
print(f'{weights.shape = }')

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(12, 12))

n_xyz = bckgr.n_xyz
n0 = bckgr.n0

plt.subplot(3, 2, 1)
plt.pcolor(xx, yy, n_fun(xx, yy, 0))
plt.axis('square')
plt.title('n_xyz')
plt.colorbar()

plt.subplot(3, 2, 2)
plt.pcolor(eta1, eta2, n0(eta1,eta2,0, squeeze_out=True).T)
plt.axis('square')
plt.title('n_0')
plt.colorbar()

make_scatter = True
if make_scatter:
    plt.subplot(3, 2, 3)
    ax = plt.gca()
    ax.set_xticks(np.linspace(0, 1, nx + 1))
    ax.set_yticks(np.linspace(0, 1., ny + 1))
    plt.tick_params(labelbottom = False) 
    coloring = weights
    plt.scatter(logpos[:, 0], logpos[:, 1], c=coloring, s=.25)
    plt.grid(c='k')
    plt.axis('square')
    plt.title('n0_scatter')
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.colorbar()

plt.subplot(3, 2, 4)
ax = plt.gca()
ax.set_xticks(np.linspace(0, 1, nx + 1))
ax.set_yticks(np.linspace(0, 1., ny + 1))
plt.tick_params(labelbottom = False) 
plt.pcolor(ee1[:,:,0], ee2[:,:,0], n_sph[:,:,0])
plt.grid()
plt.axis('square')
plt.title(f'n_sph')
plt.colorbar()

plt.subplot(3, 2, 6)
ax = plt.gca()
# ax.set_xticks(np.linspace(0, 1, nx + 1))
# ax.set_yticks(np.linspace(0, 1., ny + 1))
# plt.tick_params(labelbottom = False) 
bc_x = (be_x[:-1] + be_x[1:]) / 2. # centers of binning cells
bc_y = (be_y[:-1] + be_y[1:]) / 2.
plt.pcolor(bc_x, bc_y, f_bin)
#plt.grid()
plt.axis('square')
plt.title(f'n_binned')
plt.colorbar()

In [ ]:
from struphy.pic.sph_smoothing_kernels import linear_isotropic, trigonometric, gaussian, linear_tp

r = np.linspace(0, 1, 100)
out0 = np.zeros_like(r)
x = np.linspace(-1, 1, 200)
out1 = np.zeros_like(x)
out2 = np.zeros_like(x)
out3 = np.zeros_like(x)
for i, ri in enumerate(r):
    out0[i] = linear_isotropic(ri, 1.)
out0b = np.zeros_like(x)
out0b[:100] = out0[::-1]
out0b[100:] = out0
for i, xi in enumerate(x):
    out1[i] = trigonometric(xi, 0., 0., 1., 1., 1.)
    out2[i] = gaussian(xi, 0., 0., 1., 1., 1.)
    out3[i] = linear_tp(xi, 0., 0., 1., 1., 1.)
plt.plot(x, out0b, label="linear_isotropic")
plt.plot(x, out1, label="trigonometric")
plt.plot(x, out2, label="gaussian")
plt.plot(x, out3, label = "linear_tp")
plt.title('Some smoothing kernels')
plt.legend()

In [ ]:
from struphy.propagators.propagators_markers import PushEta

# default parameters of Propagator
opts_eta = PushEta.options(default=False)
print(opts_eta)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [ ]:
# instantiate Propagator object
prop_eta = PushEta(particles, algo = "forward_euler")

In [ ]:
from struphy.propagators.propagators_markers import PushVinSPHpressure

# default parameters of Propagator
opts_sph = PushVinSPHpressure.options(default=False)
print(opts_sph)

In [ ]:
# pass simulation parameters to Propagator class
PushVinSPHpressure.domain = domain

In [ ]:
# instantiate Propagator object
algo = "forward_euler"
kernel_width = (h1, h2, h3)
prop_v = PushVinSPHpressure(particles,
                            kernel_type = kernel_type,
                            kernel_width = kernel_width, 
                            algo = algo)

In [ ]:
import numpy as np

# time stepping
dt = 0.04
Nt = 50
Np = particles.positions.shape[0]

pos = np.zeros((Nt + 1, Np, 3), dtype=float)
velo = np.zeros((Nt + 1, Np, 3), dtype=float)
energy = np.zeros((Nt + 1, Np), dtype=float)

pos[0] = domain(particles.positions).T
velo[0] = particles.velocities

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0

while n < Nt:
    time += dt
    n += 1
    time_vec[n] = time
    
    # advance in time
    prop_eta(dt/2)
    prop_v(dt)
    prop_eta(dt/2)
    
    # positions on the physical domain Omega
    pos[n] = domain(particles.positions).T
    velo[n] = particles.velocities
    
    print(f'{n} time steps done.')

In [ ]:
plt.figure(figsize=(12, 28))
interval = Nt/10
plot_ct = 0
for i in range(Nt):
    if i % interval == 0:
        print(f'{i = }')
        plot_ct += 1
        plt.subplot(5, 2, plot_ct)
        ax = plt.gca() 
        coloring = weights
        plt.scatter(pos[i, :, 0], pos[i, :, 1], c=coloring, s=.25)
        plt.axis('square')
        plt.title('n0_scatter')
        plt.xlim(l1, r1)
        plt.ylim(l2, r2)
        plt.colorbar()
        plt.title(f'Gas at t={i*dt}')
    if plot_ct == 10:
        break